# Model Evaulation: Multiphase Maps and Extra Data

This notebook is dedicated to produce the various multiphase maps and to perform some analysis about the extra dataset employed for model evaulation

In [ ]:
#For development
#Reload the library when a change is detected in one of the imported libraries
%load_ext autoreload 
%autoreload 2 

In [ ]:
import os
import numpy as np
import pandas as pd

from script import utils, map_helper

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
kept_columns = ['Ang', 'ReL', 'FrL', 'FrG', 'X_LM', 'Eo']
model = utils.load_model('./results/Models', model_type="LightGBM")

In [ ]:
map_helper.plot_madhane_map(model, kept_columns, mesh_size=400)

In [ ]:
map_helper.plot_2d(model, kept_columns, 'Ang', np.array([90, 80, 30, 1, -1, -30, -80, -90]), mesh_size=250)

In [ ]:
map_helper.plot_3d(model, kept_columns, 'Ang', np.linspace(-90, 90, num=31), display='Flow_label', mesh_size=100)

In [ ]:
df_test = pd.read_csv('Data/Test_bronze.csv')
df_test = df_test[(df_test['Author'].str.contains('Ovad')) & (df_test['ID']==0.051) & (df_test['Ang']==0)]

In [ ]:
assert(df_test.shape[0]>0)
map_helper.test_set_comparison(model, kept_columns, df_test, display="Flow_label")

## Unseen Working Configurations:

We can now focus on evaulating the model on unseen working configurations, different from the ones of train

In [ ]:
df_secret_b = pd.read_csv('Data/Secret/Test_secret.csv')

### Accuracy Score:

The next step is to actually provide a metric to summarize the results agains unseen working configurations

In [ ]:
df_secret_g = utils.bronze_to_gold(df_secret_b, kept_columns=kept_columns+["Flow_label"])

X_secret = df_secret_g.iloc[:,:-1].values
y_secret = df_secret_g.iloc[:,[-1]].values.ravel()

In [ ]:
y_pred = model.predict(X_secret)
y_pred = np.where(y_pred==5, 1, y_pred)
y_pred = np.where(y_pred==4, 3, y_pred)

utils.print_performance(y_secret, y_pred)

In [ ]:
mi = map_helper.bar_plot_studies(df_secret_b, model, kept_columns, n_class=4)

### Probability Threshold

We can also study the effect of increasing the probability threshold on the overall accuracy

In [ ]:
df_predicted = map_helper.probability_threshold(model, X_secret, y_secret)

### Top 2 Accuracy:

It may interesting to also look at the top 2 accuracy (in order to exclude a particular regime)

In [ ]:
map_helper.get_top_two(df_predicted)